In [10]:
#资源限制 使用更轻量模型
from transformers import pipeline

model_path = "Qwen/Qwen1.5-0.5B-Chat"

generator = pipeline('text-generation',
                     model=model_path,
                     device=0,
                     do_sample=True,
                     num_return_sequences=3)

config.json:   0%|          | 0.00/661 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.24G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [13]:
generator("Good morning teacher! I have to")

[{'generated_text': 'Good morning teacher! I have to tell you that my math book is missing a few pages. Could you help me?'},
 {'generated_text': 'Good morning teacher! I have to say, your class is so much fun.'},
 {'generated_text': 'Good morning teacher! I have to ask you a question.'}]

In [16]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer


quant_path = "models/Qwen1.5-0.5B-Chat"
quant_config = {"zero_point": True, "q_group_size": 128, "w_bit": 4, "version": "GEMM"}

# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_path, device_map="cuda")
tokenizer = AutoTokenizer.from_pretrained(model_path, trust_remote_code=True)

Fetching 10 files:   0%|          | 0/10 [00:00<?, ?it/s]

In [17]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
Token indices sequence length is longer than the specified maximum sequence length for this model (8322 > 2048). Running this sequence through the model will result in indexing errors
AWQ: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 22/22 [04:41<00:00, 12.78s/it]


In [18]:
from transformers import AwqConfig, AutoConfig

# 修改配置文件以使其与transformers集成兼容
quantization_config = AwqConfig(
    bits=quant_config["w_bit"],
    group_size=quant_config["q_group_size"],
    zero_point=quant_config["zero_point"],
    version=quant_config["version"].lower(),
).to_dict()

# 预训练的transformers模型存储在model属性中，我们需要传递一个字典
model.model.config.quantization_config = quantization_config
# 保存模型权重
model.save_quantized(quant_path)
tokenizer.save_pretrained(quant_path)

('models/Qwen1.5-0.5B-Chat/tokenizer_config.json',
 'models/Qwen1.5-0.5B-Chat/special_tokens_map.json',
 'models/Qwen1.5-0.5B-Chat/chat_template.jinja',
 'models/Qwen1.5-0.5B-Chat/vocab.json',
 'models/Qwen1.5-0.5B-Chat/merges.txt',
 'models/Qwen1.5-0.5B-Chat/added_tokens.json',
 'models/Qwen1.5-0.5B-Chat/tokenizer.json')

In [22]:
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained(quant_path)
model = AutoModelForCausalLM.from_pretrained(quant_path, device_map="cuda").to(0)
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)
result = generate_text("Good afternoon teacher! Now I gonna tell you")
print(result)

Good afternoon teacher! Now I gonna tell you that the man was a businessman in his mid-40s with a tattoo on his left forearm. They both wore their regular clothing on all times.
